In [6]:
!pip install accelerate
!pip install -q --upgrade langchain
!pip install -q --upgrade langchain-openai
!pip install -q --upgrade langchain_community
!pip install "transformers==4.51.0"
!pip install -q faiss-gpu
!pip install -q pandas


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 28.1 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
!pip uninstall -y numpy
!pip install numpy==1.26.4

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os
import re

In [9]:
model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

model.safetensors.index.json:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [11]:
def classify_question(question, prev_question, prev_answer, prev_category):
    classification_prompt = f'''
당신은 반려견 상담 질문을 분류하는 전문가입니다.

사용자가 입력한 질문을 다음 세 가지 중 하나로 분류하세요:

1. 행동 교정: 반려견의 행동이 보호자에게 **불편함, 위협, 문제**로 인식되며, 그 행동을 **고치고 싶거나 줄이고 싶은 의도**가 포함된 경우
   (예: 밥 줄 때 손을 물어요, 너무 짖어요, 훈련 방법이 궁금해요 등)
2. 지식 탐색: 반려견의 습성, 특징, 돌봄 방법 등에 대해 **단순한 궁금증**을 표현한 경우
   (예: 왜 머리를 비비나요?, 눈물 자국은 왜 생기나요?, 어떤 간식을 주면 좋아하나요?)
3. 감정 공감: 반려견을 키우며 보호자가 겪는 **감정적인 어려움이나 정서적 고민**이 중심인 경우
   (예: 요즘 강아지가 버겁게 느껴져요, 너무 예뻐서 걱정돼요, 이별을 생각하면 마음이 아파요)

💡 분류 핵심 기준:
- **"왜 이러는 거야?"** 라는 표현이 있어도, 질문된 행동이 **위험하거나 교정이 필요한 행동**이면 ‘행동 교정’입니다.
- 행동 묘사 + 단순한 궁금증 = 지식 탐색
- 감정 묘사 + 고민/불편함 표현 = 감정 공감

이전 질문: {prev_question or "(없음)"}
이전 질문 분류: {prev_category or "(없음)"}
이전 응답: {prev_answer or "(없음)"}
현재 질문: {question}

📌 반드시 아래 형식으로만 출력하세요:
카테고리: 행동 교정
'''.strip()

    msgs = [{"role": "user", "content": classification_prompt}]
    prompt_text = tokenizer.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
    output = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True).strip()
    print(f"\n🧾 [모델 분류 출력]: {output}")

    match = re.search(r"카테고리\s*:\s*(행동 교정|지식 탐색|감정 공감)", output)
    if match:
        return match.group(1)
    raise ValueError(f"❌ 분류 실패: {output}")

In [12]:
from huggingface_hub import hf_hub_download

local_dir = "openai_faiss_db"

for filename in ["index.faiss", "index.pkl"]:
    hf_hub_download(
        repo_id="daaaaiin/petmind-vectorstore",
        filename=filename,
        repo_type="dataset",
        local_dir=local_dir,
        local_dir_use_symlinks=False,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


index.faiss:   0%|          | 0.00/39.9M [00:00<?, ?B/s]

index.pkl:   0%|          | 0.00/7.95M [00:00<?, ?B/s]

In [ ]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = FAISS.load_local("openai_faiss_db", embedding_model, allow_dangerous_deserialization=True)

In [14]:
PROMPT_MAP = {
    "행동 교정": """당신은 반려견 행동 문제를 상담해주는 전문가입니다.

상담의 목적은, 단순한 정보 제공이 아니라 **사용자의 상황을 정확히 이해한 뒤, 그에 맞는 맞춤형 해결책을 제시하는 것**입니다.

아래의 상담 구조를 반드시 따르세요. 이 순서를 항상 정확히 지키세요:

1. **분석**: 보호자의 고민을 바탕으로, 반려견의 행동에 대한 가능한 원인을 분석합니다.
   - 단, 절대로 추측하지 말고 입력된 정보와 문맥에 근거해서만 설명하세요.
   - 반려견 품종의 특성도 고려하세요.

2. **해결책 제시**: 분석을 기반으로 가장 유효한 1가지 해결책만 제시하세요.
   - 다양한 방법을 나열하지 말고, 상황에 맞는 핵심 조언 1가지를 간결히 전달하세요.

3. **추가 질문**: 해결책 이후, 상담을 이어가기 위한 **1개의 구체적인 질문**을 던지세요.
   - 보호자가 바로 답할 수 있도록 간단하고 상황 중심적으로 구성하세요.
   - 예) "메이가 산책 중 어떤 행동을 하나요?" 처럼 물어보세요.

❗ 절대 하지 말아야 할 것:
- 고민만 듣고 바로 해결책을 제시하지 마세요.
- 질문 없이 끝내거나, 분석 없이 해결책만 말하지 마세요.
- 같은 내용을 반복하거나 장황하게 늘어놓지 마세요.

문체 지침:
- 공감 문구는 생략하세요. 분석부터 시작하세요.
- 차분하고 전문가다운 어조로, 간결하게 작성하세요.
""",
    "지식 탐색": """당신은 반려견과 관련된 일반적인 정보를 보호자에게 이해하기 쉽게 전달하는 전문가입니다.

사용자의 질문은 반려견의 행동, 습관, 특성, 돌봄 방식 등 일상적인 궁금증에 해당하며,
당신의 역할은 **간결하고 핵심적인 정보만을 제공하여 보호자가 스스로 이해하고 판단할 수 있도록 돕는 것**입니다.

답변 지침:
- 보호자가 처음 듣는 내용도 쉽게 이해할 수 있도록, **쉬운 표현**으로 설명하세요.
- **불확실하거나 모호한 이론**은 언급하지 말고, **일반적으로 알려진 정보만** 전달하세요.
- 행동의 원인, 습성, 돌봄 팁 등은 명확히 설명하되, **훈련법이나 교정 방법은 다루지 않습니다.**
- **질병, 통증, 건강 이상 등 의학적 판단이 필요한 질문은 피하고, 반드시 수의사의 확인을 안내하세요.**

문체는 짧고 단정하게 유지하고, 정보 위주로만 구성합니다.
""",
    "감정 공감":"""
    당신은 반려견을 키우는 보호자의 감정을 이해하고, 현실적인 위로와 조언을 제공하는 감정 상담 전문가입니다.

이 역할은 반려견과의 이별, 노화 같은 특별한 순간뿐만 아니라,
양육 과정에서 느끼는 피로감, 좌절감, 거리감, 후회 등 보호자가 일상 속에서 겪는 감정적 어려움까지도 다룹니다.

답변 목적:
- 감정 표현에 공감하는 데 그치지 않고, 그 감정의 원인을 함께 찾고 이해할 수 있도록 도와주는 것입니다.
- 감정의 원인이 질문 속에 명확히 드러나지 않은 경우, 사용자가 스스로 감정을 정리할 수 있도록 **추가 질문을 통해 유도**하세요.
- 감정을 탐색하고 해소할 수 있도록, 상담자처럼 대화를 이끌어가야 합니다.

답변 구조:
1. 보호자의 감정 표현에 진심 어린 공감
2. 감정의 원인이 명확하다면 → 이를 간결히 정리하고 감정 수용
3. 감정의 원인이 불분명하다면 → 추가 질문 1~2개를 통해 이유를 함께 탐색
4. 감정을 정리하고, 반려견과의 일상으로 다시 연결될 수 있도록 가볍고 현실적인 조언 제시

문체 지침:
- 지나치게 감성적인 문장, 장황한 설명은 피하고, 따뜻하면서도 차분한 어조를 유지하세요.
- 위로는 현실적이어야 하며, 보호자가 부담을 느끼지 않도록 간결하게 말하세요.
- 반려견은 절대로 '그녀', '그'처럼 인격화하지 말고, 반드시 '반려견', '강아지'처럼 중립적이거나 반려견 이름으로 지칭하세요.
"""
}


In [15]:
def get_user_input():
    user_input = input("\n🧑 사용자 고민: ").strip()
    if "완료" in user_input:
        print("\n✅ 대화를 종료합니다.")
        return None
    return user_input

def classify_and_get_prompt(user_input, prev_q, prev_a, prev_cate):
    category = classify_question(user_input, prev_q, prev_a, prev_cate)
    print(f"\n📌 분류된 카테고리: {category}")
    prompt = PROMPT_MAP[category]
    return category, {"role": "system", "content": prompt}

def search_documents(user_input):
    retrieved_docs_with_score = vectordb.similarity_search_with_score(user_input, k=3)
    threshold = 1.0
    filtered_docs = [
        doc.page_content
        for doc, score in retrieved_docs_with_score
        if score <= threshold
    ]

    if filtered_docs:
        retrieved_context = "\n\n".join(filtered_docs)
        print("🔍 검색된 문서:\n", retrieved_context)
        return retrieved_context
    else:
        print("⚠️ 유사한 문서가 없습니다.")
        return None

def build_chat_messages(system_msg, user_input, dog_info, chat_history, is_first, category, context=None):
    if context:
        system_msg["content"] = f"{system_msg['content']}\n\n{context}"  # RAG 내용 추가

    dog_name = dog_info.get("name", "반려견")
    dog_breed = dog_info.get("breed", "강아지")
    dog_age = dog_info.get("age", "알 수 없음")
    gender = dog_info.get("gender", "모름")
    neutered = dog_info.get("neutered", "모름")
    disease = dog_info.get("disease", "모름")
    disease_desc = dog_info.get("disease_desc", "")
    period = dog_info.get("period", "")
    housing = dog_info.get("housing", "")

    dog_profile = (
        f"• 이름: {dog_name}\n"
        f"• 견종: {dog_breed}\n"
        f"• 나이: {dog_age}살\n"
        f"• 성별: {gender}\n"
        f"• 중성화 여부: {neutered}\n"
        f"• 질병 이력: {disease} {'- ' + disease_desc if disease == '있음' else ''}\n"
        f"• 함께 산 기간: {period}\n"
        f"• 주거 형태: {housing}"
    )

    user_message = (
        f"[보호자 질문]\n{user_input}\n\n"
        f"[반려견 프로필]\n{dog_profile}"
    )

    messages = [system_msg]

    if is_first:
        greeting = f"안녕하세요. {dog_name} 보호자님! 상담을 시작하겠습니다."
        messages.append({"role": "assistant", "content": greeting})
        chat_history.append({"role": "assistant", "content": greeting})

    messages += chat_history[-8:]
    messages.append({"role": "user", "content": user_message})

    print("\n[DEBUG] 🧩 build_chat_messages 호출됨")
    print("[DEBUG] system_msg:", system_msg)
    print("[DEBUG] user_input:", user_input)
    print("[DEBUG] chat_history (최신 8개):", chat_history[-8:])
    print("[DEBUG] 최종 messages 길이:", len(messages))
    print("[DEBUG] 반환 타입 확인:", type(messages), type(chat_history))

    return messages, chat_history


def generate_response(messages):
    prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=True)
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            temperature=0.6,
            top_p=0.95,
            top_k=20,
            do_sample=True
        )
    return outputs[0][inputs.input_ids.shape[-1]:].tolist(), inputs.input_ids.shape[-1]

def split_thinking_and_content(output_ids, input_len):
    try:
        end_token_id = 151668  # </think>
        index = len(output_ids) - output_ids[::-1].index(end_token_id)
    except ValueError:
        index = 0
    thinking = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip()
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()
    return thinking, content

def print_response(category, content):
    print(f"\n🤖 [{category} 응답]: {content}")


In [16]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 83.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 103.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 28.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip ins

In [17]:
import gradio as gr

# 견종 리스트
BREED_LIST = [
    "말티즈", "푸들", "포메라니안", "요크셔테리어", "시바견",
    "리트리버", "웰시코기", "치와와", "비숑프리제", "닥스훈트"
]

def save_dog_profile(img, name, breed, age, gender, neutered, disease, disease_desc, period, housing):

    if not name or not breed:
        return (
            gr.update(visible=True, value="❌ 이름과 견종은 필수 입력입니다."),
            gr.update(selected="반려견 정보 입력"),
            gr.update(value={}),
            gr.update(value=[])
        )

    profile = {
        "img": img,
        "name": name,
        "breed": breed,
        "age": age,
        "gender": gender,
        "neutered": neutered,
        "disease": disease,
        "disease_desc": disease_desc if disease == "있음" else "",
        "period": period,
        "housing": housing,
        "chat_history": [],
        "prev_q": None,
        "prev_a": None,
        "prev_cate": None,
        "is_first_question": True,
    }

    greeting = f"안녕하세요! {name} 보호자님! ({breed})에 대해 상담을 시작해보겠습니다."
    profile["chat_history"].append({"role": "assistant", "content": greeting})

    return (
        gr.update(visible=True, value=f"✅ {name} ({breed}) 정보가 저장되었습니다."),
        gr.update(selected="고민 상담하기"),
        profile, 
        profile["chat_history"]
    )

def consult(user_msg, profile):
    # ✅ 카테고리 분류 및 프롬프트 구성 함수 사용
    category, system_msg = classify_and_get_prompt(
        user_msg,
        profile.get("prev_q"),
        profile.get("prev_a"),
        profile.get("prev_cate")
    )

    # ✅ 문서 검색 (RAG)
    context = search_documents(user_msg)

    # ✅ 메시지 구성
    chat_history = profile.get("chat_history", [])
    is_first = profile.get("is_first_question", True)

    messages, _ = build_chat_messages(
        system_msg, user_msg, profile, chat_history, is_first, category, context=context
    )

    # ✅ 응답 생성
    output_ids, input_len = generate_response(messages)
    _, content = split_thinking_and_content(output_ids, input_len)

    # ✅ 채팅 기록 업데이트
    chat_history.append({"role": "user", "content": user_msg})
    chat_history.append({"role": "assistant", "content": content})

    # ✅ 프로필 업데이트
    profile.update({
        "chat_history": chat_history,
        "prev_q": user_msg,
        "prev_a": content,
        "prev_cate": category,
        "is_first_question": False
    })

    return "", chat_history, profile


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    dog_profile_state = gr.State({})

    with gr.Tabs(selected="반려견 정보 입력") as tabs:
        with gr.Tab("반려견 정보 입력"):
            gr.Markdown("## 🐶 반려견 정보 입력")
            gr.Markdown("📷 반려견의 사진을 등록해주세요.")
            with gr.Row():
                with gr.Column():
                    dog_img = gr.Image(label="반려견 사진", type="pil", sources=["upload"])
                with gr.Column():
                    dog_name = gr.Textbox(placeholder="예: 아리, 보리, 뭉치", label="반려견 이름")
                    dog_breed = gr.Dropdown(choices=BREED_LIST, label="반려견 견종", allow_custom_value=True)
                    dog_age = gr.Number(label="나이 (살)", precision=0)
                    dog_gender = gr.Radio(["남아", "여아"], label="성별", value=None)
                    dog_neutered = gr.Radio(["중성화 완료", "중성화 미완료", "모름"], label="중성화 여부", value=None)
                    dog_disease = gr.Radio(["있음", "없음"], label=" 과거 질병/수술 이력 여부", value=None)
                    dog_disease_detail = gr.Textbox(label="질병/수술 상세 내용", placeholder="예: 슬개골 탈구 수술 (2022)")
                    dog_period = gr.Radio(["① 1년 미만", "② 1년 이상~3년 미만", "③ 3년 이상~10년 미만", "④ 10년 이상"], label="함께 산 기간", value=None)
                    dog_housing = gr.Dropdown(choices=["아파트", "단독주택", "빌라/다세대", "기타"], label="주거 형태", value=None)

            confirm_button = gr.Button("✅ 반려견 정보 확인")
            confirm_msg = gr.Markdown("", visible=False)

        with gr.Tab("고민 상담하기"):
            chatbot_ui = gr.Chatbot(label="상담 챗봇", type="messages")
            user_msg_input = gr.Textbox(label="보호자 고민 입력", placeholder="예: 산책 중에 자꾸 짖어요", scale=8)
            send_button = gr.Button("상담 전송")

    confirm_button.click(
        fn=save_dog_profile,
        inputs=[
            dog_img, dog_name, dog_breed, dog_age, dog_gender,
            dog_neutered, dog_disease, dog_disease_detail,
            dog_period, dog_housing
        ],
        outputs=[
            confirm_msg, tabs, dog_profile_state, chatbot_ui
        ]
    )

    send_button.click(
        fn=consult,
        inputs=[user_msg_input, dog_profile_state],
        outputs=[user_msg_input, chatbot_ui, dog_profile_state]
    )

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://406f66d9268ba143fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🧾 [모델 분류 출력]: 카테고리: 행동 교정

📌 분류된 카테고리: 행동 교정
🔍 검색된 문서:
 수 있습니다. 우리 강아지, 추위 타고 있을까요? 강아지가 추위를 느끼고 있다면, 아래와 같은 행동을 할 수 있습니다. 몸을 떤다 산책 중 걷지 않는다 산책 중 건물로 들어가려 한다 낑낑거린다 꼬리를 다리 사이로 집어넣고 등을 구부린다 쿠팡 파트너스 활동을 통해 일정액의 수수료를 제공받을 수 있음 관련 제품

밟아보고, 아스팔트 냄새도 맡아보고 다른 친구들과 어울려 사회성을 기른다면 강아지와 행복한 하루하루를 보낼 수 있겠죠? 또한, 예방접종이 덜 끝나서 전염병 등이 걱정된다면 강아지를 안고서라도 자주 산책하는 것도 추천 드립니다. 4. 위험한 환경에서는 안아서 이동하기 강아지와 산책 중에 예상치 못한 일들이 일어날 수 있기 때문에, 항상 주의깊게 살펴야합니다. 골목 귀퉁이를 도는데 갑자기 차가 들어올 수 있고, 크고 작은 크기의 날카로운 유리 파편들이 바닥에 있을 수도 있으며, 다른 동물의 배변을 먹을 수도 있어요. 언제 어디서 차가 들어올지 모르는 골목에서는 안고 이동을 하거나 줄을 팽팽하게 당긴채로 걷는 것이 좋습니다. 유리나 작은 돌이 많은 길을 걸어야할 때는, 안은 채로 지나가거나 강아지에게 바닥이 튼튼한 신발을 신기는 것도 좋겠죠? 마지막으로, 다른 동물의 배변을 먹는 것은 아주 위험하기 때문에 섭취하지 못하도록 안 돼 키워드와 함께 빠르게 이동을 하는 것이 좋습니다. 여기까지 산책 훈련에 대해서

[DEBUG] 🧩 build_chat_messages 호출됨
[DEBUG] system_msg: {'role': 'system', 'content': '당신은 반려견 행동 문제를 상담해주는 전문가입니다.\n\n상담의 목적은, 단순한 정보 제공이 아니라 **사용자의 상황을 정확히 이해한 뒤, 그에 맞는 맞춤형 해결책을 제시하는 것**입니다.\n\n아래의 상담 구조를 반드시 따르세요. 이 순서를 항상 정확히 지키세요:\n\n1. **분석**: 보호자의 고민을